In [ ]:
!pip install scrapy --quiet
!pip install crochet --quiet

     |████████████████████████████████| 271 kB 4.8 MB/s 
     |████████████████████████████████| 57 kB 4.8 MB/s 
     |████████████████████████████████| 93 kB 2.1 MB/s 
     |████████████████████████████████| 4.0 MB 53.9 MB/s 


In [ ]:
# scrape webpage
import scrapy
from scrapy.crawler import CrawlerRunner
# text cleaning
import re
# Reactor restart
from crochet import setup, wait_for 
setup()

In [ ]:
class Item(scrapy.Item):
    Brand = scrapy.Field()
    Model = scrapy.Field()
    Seat = scrapy.Field()
    Battery = scrapy.Field()
    Range = scrapy.Field()
    Topspeed = scrapy.Field()
    Efficiency = scrapy.Field()
    Fastcharge= scrapy.Field()
    Acceleration = scrapy.Field()
    Tow_Capacity = scrapy.Field()
    Drive = scrapy.Field()
    Price_GE = scrapy.Field()
    Price_Nl = scrapy.Field()
    Price_UK = scrapy.Field()
    Market_Segment = scrapy.Field()

class QuotesToCsv(scrapy.Spider):
    """scrape first line of  quotes from ```wikiquote``` by 
    Maynerd James Keenan and save to json file"""
    name = "EV_BASE"
    start_urls = [
        'https://ev-database.org',
    ]
    custom_settings = {
        'REQUEST_FINGERPRINTER_IMPLEMENTATION' : '2.7',

        'FEEDS': {
            'EV_BASE.csv': {
                'format': 'csv',
                'overwrite': True
            }
        }
    }

    def parse(self, response):
        """parse data from urls"""
        for i in response.xpath('.//div[@class="data-wrapper"]'):
            item = Item()
            item['Brand'] = i.css('span::text').get()
            item['Model'] = i.css('span.model::text').get()
            item['Seat'] =  i.xpath('.//*[@title="Number of seats"]/text()').get()
            item['Battery'] = i.css('span.battery::text').get()
            item['Range'] = i.css('span.erange_real::text').get()
            item['Topspeed'] = i.css('span.topspeed::text').get()
            item['Efficiency'] = i.css('span.efficiency::text').get()
            item['Fastcharge'] = i.css('span.fastcharge_speed_print::text').get()
            item['Acceleration'] = i.css('span.acceleration::text').get()
            item['Tow_Capacity'] = i.xpath('.//*[@title="Towing capacity in kg"]/text()').get()
            item['Drive'] = i.xpath('.//*[@class="icons"]/span[4]/@title').get()
            if item['Drive'] is None:
                item['Drive'] = i.xpath('.//*[@class="icons"]/span[2]/@title').get()
            item['Market_Segment'] = i.xpath('.//*[@title="Market Segment"]/text()').get()
            item['Price_GE'] = i.css('span.country_de::text').get()
            item['Price_Nl'] = i.css('span.country_nl::text').get()
            item['Price_UK'] = i.css('span.country_uk::text').get()
            yield item

@wait_for(10)
def run_spider():
    """run spider with MJKQuotesToCsv"""
    crawler = CrawlerRunner()
    d = crawler.crawl(QuotesToCsv)
    return d
run_spider()

In [ ]:
import pandas as pd 
df = pd.read_csv('EV_BASE.csv')
df.head(5)

,Acceleration,Battery,Brand,Drive,Efficiency,Fastcharge,Market_Segment,Model,Price_GE,Price_Nl,Price_UK,Range,Seat,Topspeed,Tow_Capacity
0,3.5 sec,74.0,Kia,All Wheel Drive,200 Wh/km,970 km/h,C,EV6 GT,"€69,990","€69,295","£61,595",370 km,5,260 km/h,1800.0
1,5.0 sec,75.0,Tesla,All Wheel Drive,172 Wh/km,670 km/h,D,Model Y Long Range Dual Motor,"€59,965","€66,995","£57,990",435 km,5,217 km/h,1600.0
2,6.1 sec,57.5,Tesla,Rear Wheel Drive,151 Wh/km,630 km/h,D,Model 3,"€52,965","€52,995","£48,490",380 km,5,225 km/h,1000.0
3,4.4 sec,75.0,Tesla,All Wheel Drive,155 Wh/km,750 km/h,D,Model 3 Long Range Dual Motor,"€62,465","€60,995","£57,490",485 km,5,233 km/h,1000.0
4,6.9 sec,57.5,Tesla,Rear Wheel Drive,167 Wh/km,580 km/h,D,Model Y,"€56,965","€50,995","£51,990",345 km,5,217 km/h,1600.0
